In [ ]:
import xarray as xr
import numpy as np
import matplotlib as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
from io import BytesIO
from PIL import Image, ImageDraw

Lets gooooooo

Troubleshooting for netcdf: 

In [ ]:
def pull_plot(region, start_date, end_date):
    
    
    """ This function takes the input region, start date, and end date, obtains the satellite 
        data from the ssec.wisc.edu ftp server and generates an output .gif saved to the local 
        machine. It does not store files temporarily on local machine. 
    
        region: string, input region from bulleted list below
        start_date: string of date to start loop featuring day, month, and year in any order using either / or - 
        end_date: string of date to start loop featuring day, month, and year in any order using either / or -     
    
    """
    
    
    
    date_range = pd.date_range(start = start_date, end = end_date, freq = 'H')                 ## establish date range given inputs 
    months = pd.unique([i.strftime('%Y%m') for i in date_range])                               ## create unique Year/Month array to loop through ftp directory based on years and months
                                                                                               ## this is somewhat of a saftey net if event desires straddles months or years
    
    ftp = ftplib.FTP('ftp.ssec.wisc.edu', 'anonymous')                                         ## log into the FTP server

    images = []                                                                                ## create empty array to store files obtained from looping through ftp directories

    for yearmon in months:                                                                     ## loop through unique years/months desired
            
        try:
       
            ftp.cwd(f"/pub/mtpw2/images/tpw_nrl_colors/{region}/{yearmon}")                    ## cd into correct directory based on region and year/month desired
        
            for pic in sorted(ftp.nlst()):                                                     ## ftp.nlst lists all the files in our ftp directory accessed above, we are looping over each image in this list

                r = BytesIO()                                                                  ## establish an in-memory bytes buffer for us to temporarily store the ftp image in

                sub = pd.to_datetime(str(pic)[4:15], format = '%Y%m%d.%H')                     ## ensure the image selected is within date range

                if sub >= date_range[0] and sub <= date_range[-1]:

                    print(f'{sub} is downloading')

                    ftp.retrbinary(f'RETR {pic}', r.write)                                     ## retrive the image from the list and store it in our established byte buffer

                    images.append(r)                                                           ## append the byte buffer for later plotting
        
        except:

            print('Invalid region or date range inputted')
            
        
    img, *imgs = [Image.open(f) for f in images]                                               ## formally open each temporarily stored byte buffer in images list
    
    img.save(f'IWV_{region}_{str(date_range[0])[0:10]}_{str(date_range[-1])[0:10]}.gif',       ## utilize image gif plotter that accepts output name (first row here is the saved output name based on region and dates)
             format='GIF', append_images=imgs, save_all=True, duration=200, loop=3)            ## and append_images = our list of opened images that are in byte form
                                                                                               ## gif features can be altered here such as # of loops and speed     

    ftp.quit()                                                                                 ## make sure to close our connection!! 



In [ ]:
## example of inputs for pull_plot function
## note how start and end date 

region = 'samer2'
start_date = '2018-12-31'
end_date = '2019-01-01'

In [ ]:
%%time

pull_plot(region, start_date, end_date)